In [1]:
!pip install pandas altair altair_data_server

In [94]:
import pandas
import altair

altair.data_transformers.enable('data_server')

DataTransformerRegistry.enable('data_server')

In [95]:
data = pandas.read_json('use_numpy_api.jsonl', lines=True)
data

,function,params
0,numpy.arange,{'0': 10}
1,_operator.pos,"{'a': {'__tp': 'numpy.ndarray', '__v': {'shape..."
2,_operator.neg,"{'a': {'__tp': 'numpy.ndarray', '__v': {'shape..."
3,_operator.invert,"{'a': {'__tp': 'numpy.ndarray', '__v': {'shape..."
4,_operator.not_,"{'a': {'__tp': 'numpy.ndarray', '__v': {'shape..."
...,...,...
69,builtins.getattr,"{'0': {'__tp': 'numpy.ndarray', '__v': {'shape..."
70,numpy.ndarray.reshape,"{'0': {'__tp': 'numpy.ndarray', '__v': {'shape..."
71,numpy.column_stack,"{'tup': [{'__tp': 'numpy.ndarray', '__v': {'sh..."
72,builtins.getattr,"{'0': {'__tp': 'builtins.module', '__v': 'nump..."


In [96]:
altair.Chart(data).mark_bar().encode(
    y=altair.Y('function', sort='-x'),
    x='count()'
)

alt.Chart(...)

In [97]:
def get_args(fn_name):
    return pandas.DataFrame.from_records(list(data[data.function == fn_name]['params'].to_dict().values()))

In [98]:
get_args('_operator.getitem')

,a,b
0,"{'__tp': 'numpy.ndarray', '__v': {'shape': [10...",0
1,"{'__tp': 'numpy.ndarray', '__v': {'shape': [10...","{'__tp': 'numpy.ndarray', '__v': {'shape': [10..."
2,"{'__tp': 'numpy.ndarray', '__v': {'shape': [10...",0
3,"{'__tp': 'numpy.ndarray', '__v': {'shape': [10...","{'__tp': 'builtins.slice', '__v': [1, 2, None]}"
4,"{'__tp': 'numpy.ndarray', '__v': {'shape': [5,...","[[1, 2, 3], [0, 1, 1]]"


In [69]:
get_types('numpy.zeros')

,0
0,"[4, 5]"


In [70]:
get_types('numpy.linspace').head()

,0,1,2
0,0,2,100
